In [1]:
import ipyparallel as ipp

rc = ipp.Client(profile="bluefog")
dview = rc[:]  # A DirectView of all engines
dview.block = True
rc.ids

[0, 1, 2, 3]

In [2]:
%%px
import bluefog.torch as bf

bf.init()
message = f"Hello, I am {bf.rank()} among {bf.size()} processes"
print(message)

[stdout:3] Hello, I am 0 among 4 processes


[stdout:1] Hello, I am 2 among 4 processes


[stdout:0] Hello, I am 1 among 4 processes


[stdout:2] Hello, I am 3 among 4 processes


[stderr:1] Invalid MIT-MAGIC-COOKIE-1 key

[stderr:0] Invalid MIT-MAGIC-COOKIE-1 key

[stderr:3] Invalid MIT-MAGIC-COOKIE-1 key

[stderr:2] Invalid MIT-MAGIC-COOKIE-1 key

In [3]:
%%px

import numpy as np
import scipy.linalg as la
""" Manifold package, Stiefel manifold and Euclidean space """
__license__ = 'MIT'
__author__ = 'Shixiang Chen'
__email__ = 'chenshxiang@gmail.com'


class StiefelManifold:
    # def __int__(self):
    @staticmethod     # 函数装饰器，定义了这个可以在不创立类的实例的情况下调用函数内部的方法
    def proj_manifold(x):   # orthogonal projection onto the manifold
        u, s, vh = la.svd(x, full_matrices=False)
        return np.matmul(u, vh)  #返回矩阵乘积


    @staticmethod
    def proj_tangent(x, d):
        xd = np.matmul(x.T,  d)
        pd = d - 0.5 * np.matmul(x, xd + xd.T)   #这是将欧氏空间的梯度方向投影到流形的切空间中
        return pd

    # 基于QR分解的收缩映射
    @staticmethod
    def qrretraction(x, d):
        Q,R=la.qr(x + d,mode='economic')
        return Q
    
    @staticmethod
    def retraction(x, d):    
        u, s, vh = la.svd(x + d, full_matrices=False)    #这应该就是ret函数
        return np.matmul(u, vh)

    # 基于polar 的收缩映射
    @staticmethod
    def repolar(x,d):
        a=x + d
        b=np.matmul(d.T,d)+np.identity(min(d.shape))
        b=1
        return np.matmul(a,b)

    @staticmethod  
    def generate_stiefel_matrix(m, n):
        A = np.random.randn(m, n)
        Q, R = np.linalg.qr(A)
        return Q[:, :n]

In [5]:
import sys 
print(sys.path)

['/home/yl-02/下载/experiments-main', '/home/yl-02/anaconda3/envs/bluefog_env/lib/python38.zip', '/home/yl-02/anaconda3/envs/bluefog_env/lib/python3.8', '/home/yl-02/anaconda3/envs/bluefog_env/lib/python3.8/lib-dynload', '', '/home/yl-02/.local/lib/python3.8/site-packages', '/home/yl-02/anaconda3/envs/bluefog_env/lib/python3.8/site-packages', '..']


In [4]:
#create data
import torch
import sys 
sys.path.append("..") 
from data_generate import *
from ManifoldTool import StiefelManifold

M = StiefelManifold()

features = 8   #n
componments = 3   #r
samples = 10000
eigengap = 0.8

#数据分配
data = synthetic_data(features, samples, eigengap)
num_nodes = len(rc.ids)
num_split = int(samples // num_nodes)
for i in range(num_nodes):
    data_worker = data.T[i * num_split : (i + 1) * num_split]
    data_worker = data_worker.T
    dview.push({"data_worker": data_worker}, targets=i)
    

#x初始化
x_0 = torch.randn(features, componments, dtype=torch.float)
print(x_0)
x_init = M.retraction(x_0, 0)


#计算x*为ite_opt
_, opt = PCA(data, componments)
#print("opt = {}".format(opt))

x_opt, loss_opt = ite_PCA(data, x_init)
#print("x_opt = {}".format(x_opt))

dview.push({"x_init": x_init})
dview.push({"x_opt": x_opt})

ModuleNotFoundError: No module named 'data_generate'

In [5]:
%%px

print("I received value: ", x_opt, x_init)
print(data_worker.shape)

[0:execute]
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[8], line 1
----> 1 print("I received value: ", x_opt, x_init)
      2 print(data_worker.shape)

NameError: name 'x_opt' is not defined
[1:execute]
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[8], line 1
----> 1 print("I received value: ", x_opt, x_init)
      2 print(data_worker.shape)

NameError: name 'x_opt' is not defined
[3:execute]
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[8], line 1
----> 1 print("I received value: ", x_opt, x_init)
      2 print(data_worker.shape)

NameError: name 'x_opt' is not defined
[2:execute]
-------------------------------------------------------------------

AlreadyDisplayedError: 4 errors

In [ ]:
%%px

import os
import numpy as np
#import bluefog.torch as bf
import torch
from bluefog.common import topology_util
import networkx as nx
import matplotlib.pyplot as plt

import time
from datetime import datetime
import pytz
import pickle

import sys 
sys.path.append("..") 
from graph import *
from data_generate import *
from ManifoldTool import StiefelManifold

%matplotlib inline

M = StiefelManifold()

torch.set_printoptions(precision=8)

maxite = 1000
alpha = 1 #代表着趋向共识的权重
lamda = 0.01 #代表着迭代的步长 
componments = 3   #r
features = 8   #n
samples = 2500
eigengap = 0.8


taskname = os.path.join('./figure')
os.makedirs(taskname, exist_ok=True)
#初始化
k = bf.rank()    #number of nodes
graph_type = 'Star'
G = graph_make(graph_type)
bf.set_topology(G)


In [ ]:
%%px

def Stochastic_gta_one_step(x, y, prev_g, x_opt, data, beta, ite):
    
    v = M.proj_tangent(x, y)

    if not torch.is_tensor(x):
        x = torch.tensor(x)
    x= M.retraction(x, lamda*(M.proj_tangent(x, alpha * bf.neighbor_allreduce(x)) + beta * v))

    if not torch.is_tensor(x):
        x = torch.tensor(x)

    batchsize = min([samples-1,int(ite + 1)])
    idx = generate_random_array(batchsize, samples)
    w = data[:, idx]
    w = torch.from_numpy(w)  # 将 numpy 数组转换成 Tensor
    g = gradient(x, w)

    if not torch.is_tensor(y):
        y = torch.tensor(y)
    trac = g - prev_g
    q = np.add(y, trac)
    y = bf.neighbor_allreduce(q)
    
    x_mean = bf.allreduce(x)  #算术平均
    x_mean = M.proj_manifold(x_mean)  #IAM
    if not torch.is_tensor(x_mean):
        x_mean = torch.tensor(x_mean)
  
    rel_error = sub_dist(x_mean, x_opt)
    
    return x, y, g, rel_error

def SGD_one_step(x, x_opt, data, beta_0, ite):
    beta = beta_0 / np.sqrt(ite+1)
    batchsize = 1
    
    idx = generate_random_array(batchsize, samples)
    w = data[:, idx]
    w = torch.from_numpy(w)  # 将 numpy 数组转换成 Tensor
    g = gradient(x, w)

    
    if not torch.is_tensor(x):
        x = torch.tensor(x)
    x = M.retraction(x, M.proj_tangent(x, alpha * bf.neighbor_allreduce(x)) + beta * g)

    if not torch.is_tensor(x):
        x = torch.tensor(x)
    
    x_mean = bf.allreduce(x)  #算术平均
    x_mean = M.proj_manifold(x_mean)  #IAM
    if not torch.is_tensor(x_mean):
        x_mean = torch.tensor(x_mean)
  
    error = sub_dist(x_mean, x_opt)
    
    return x, error
    
#初始化
x_1 = x_init
y = x_init
prev_g = x_init
rel_error_1= torch.zeros((maxite, 1))

x_2 = x_init
rel_error_2= torch.zeros((maxite, 1))

for i in range(maxite):
    # seed = 1212
    # torch.manual_seed(seed)

    x_1, y, prev_g, rel_error_1[i] = Stochastic_gta_one_step(x_1, y,prev_g, x_opt, data_worker, beta = 0.08, ite = i)
    x_2, rel_error_2[i] = SGD_one_step(x_2, x_opt, data_worker, beta_0 = 0.1, ite = i)

    if bf.rank() == 0:     #第1个client的结果
        if(i % 500 == 0):
            print(i, rel_error_1[i])

# bf.shutdown()


In [ ]:
error_1_all = dview.pull("rel_error_1", block=True, targets=0)
error_2_all = dview.pull("rel_error_2", block=True, targets=0)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

plt.semilogy(error_1_all)
plt.semilogy(error_2_all)

plt.legend(["Alg:1", "DR-SGD"], fontsize=16)

plt.xlabel("Iteration", fontsize=16)
plt.ylabel("Optimal error", fontsize=16)

plt.savefig( 'optimal——err.png')